In [1]:
import copy

import pytorch_lightning as pl
import torch
import torch.nn as nn
import torchvision

from lightly.data import LightlyDataset, SimCLRCollateFunction, collate
from lightly.loss import NTXentLoss
from lightly.models import ResNetGenerator
from lightly.models.modules.heads import MoCoProjectionHead
from lightly.models.utils import (
    batch_shuffle,
    batch_unshuffle,
    deactivate_requires_grad,
    update_momentum,
)

C:\Users\isxzl\anaconda3\envs\autoGPT\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
num_workers = 8
batch_size = 512
memory_bank_size = 4096
seed = 1
max_epochs = 1
input_size=32

In [3]:
path_to_train = "../Datasets/cifar10/train/"
path_to_test = "../Datasets/cifar10/test/"

In [4]:
pl.seed_everything(seed)

Global seed set to 1


1

In [5]:
# MoCo v2 uses SimCLR augmentations, additionally, disable blur
collate_fn = SimCLRCollateFunction(input_size=input_size, vf_prob=0.5, rr_prob=0.5)

In [6]:
# Augmentations typically used to train on cifar-10
train_classifier_transforms = torchvision.transforms.Compose(
    [
        torchvision.transforms.RandomCrop(input_size, padding=4),
        torchvision.transforms.RandomHorizontalFlip(),
        torchvision.transforms.ToTensor(),
        torchvision.transforms.Normalize(
            mean=collate.imagenet_normalize["mean"],
            std=collate.imagenet_normalize["std"],
        ),
    ]
)

# No additional augmentations for the test set
test_transforms = torchvision.transforms.Compose(
    [
        torchvision.transforms.Resize((input_size, input_size)),
        torchvision.transforms.ToTensor(),
        torchvision.transforms.Normalize(
            mean=collate.imagenet_normalize["mean"],
            std=collate.imagenet_normalize["std"],
        ),
    ]
)

In [7]:

# We use the moco augmentations for training moco
dataset_train_moco = LightlyDataset(input_dir=path_to_train)

# Since we also train a linear classifier on the pre-trained moco model we
# reuse the test augmentations here (MoCo augmentations are very strong and
# usually reduce accuracy of models which are not used for contrastive learning.
# Our linear layer will be trained using cross entropy loss and labels provided
# by the dataset. Therefore we chose light augmentations.)
dataset_train_classifier = LightlyDataset(
    input_dir=path_to_train, transform=train_classifier_transforms
)

dataset_test = LightlyDataset(input_dir=path_to_test, transform=test_transforms)

In [8]:
dataloader_train_moco = torch.utils.data.DataLoader(
    dataset_train_moco,
    batch_size=batch_size,
    shuffle=True,
    collate_fn=collate_fn,
    drop_last=True,
    num_workers=num_workers,
)

dataloader_train_classifier = torch.utils.data.DataLoader(
    dataset_train_classifier,
    batch_size=1,
    shuffle=True,
    drop_last=True,
    num_workers=num_workers,
)

dataloader_test = torch.utils.data.DataLoader(
    dataset_test,
    batch_size=1,
    shuffle=False,
    drop_last=False,
    num_workers=num_workers,
)

In [9]:
class MocoModel(pl.LightningModule):
    def __init__(self):
        super().__init__()

        # create a ResNet backbone and remove the classification head
        resnet = ResNetGenerator("resnet-18", 1, num_splits=8)
        self.backbone = nn.Sequential(
            *list(resnet.children())[:-1],
            nn.AdaptiveAvgPool2d(1),
        )

        # create a moco model based on ResNet
        self.projection_head = MoCoProjectionHead(512, 512, 128)
        self.backbone_momentum = copy.deepcopy(self.backbone)
        self.projection_head_momentum = copy.deepcopy(self.projection_head)
        deactivate_requires_grad(self.backbone_momentum)
        deactivate_requires_grad(self.projection_head_momentum)

        # create our loss with the optional memory bank
        self.criterion = NTXentLoss(temperature=0.1, memory_bank_size=memory_bank_size)

    def training_step(self, batch, batch_idx):
        (x_q, x_k), _, _ = batch

        # update momentum
        update_momentum(self.backbone, self.backbone_momentum, 0.99)
        update_momentum(self.projection_head, self.projection_head_momentum, 0.99)

        # get queries
        q = self.backbone(x_q).flatten(start_dim=1)
        q = self.projection_head(q)

        # get keys
        k, shuffle = batch_shuffle(x_k)
        k = self.backbone_momentum(k).flatten(start_dim=1)
        k = self.projection_head_momentum(k)
        k = batch_unshuffle(k, shuffle)

        loss = self.criterion(q, k)
        self.log("train_loss_ssl", loss)
        return loss

    #def on_train_epoch_end(self):
     #   self.custom_histogram_weights()

    # We provide a helper method to log weights in tensorboard
    # which is useful for debugging.
  #  def custom_histogram_weights(self):
    #    for name, params in self.named_parameters():
    #        self.logger.experiment.add_histogram(name, params, self.current_epoch)

    def configure_optimizers(self):
        optim = torch.optim.SGD(
            self.parameters(),
            lr=6e-2,
            momentum=0.9,
            weight_decay=5e-4,
        )
        scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optim, max_epochs)
        return [optim], [scheduler]

In [12]:
model = MocoModel()
trainer = pl.Trainer(max_epochs=max_epochs, devices=1, accelerator="gpu")
trainer.fit(model, dataloader_train_moco)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name                     | Type               | Params
----------------------------------------------------------------
0 | backbone                 | Sequential         | 11.2 M
1 | projection_head          | MoCoProjectionHead | 328 K 
2 | backbone_momentum        | Sequential         | 11.2 M
3 | projection_head_momentum | MoCoProjectionHead | 328 K 
4 | criterion                | NTXentLoss         | 0     
----------------------------------------------------------------
11.5 M    Trainable params
11.5 M    Non-trainable params
23.0 M    Total params
91.977    Total estimated model params size (MB)


Epoch 0: 100%|███████████████████████████████████████████████████████████████| 97/97 [01:12<00:00,  1.34it/s, v_num=14]

`Trainer.fit` stopped: `max_epochs=1` reached.


Epoch 0: 100%|███████████████████████████████████████████████████████████████| 97/97 [01:12<00:00,  1.33it/s, v_num=14]


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type             | Params
-----------------------------------------------
0 | backbone  | Sequential       | 11.2 M
1 | fc        | Linear           | 5.1 K 
2 | criterion | CrossEntropyLoss | 0     
-----------------------------------------------
5.1 K     Trainable params
11.2 M    Non-trainable params
11.2 M    Total params
44.696    Total estimated model params size (MB)


In [ ]:
from sklearn import svm
import numpy as np
import torch
from tqdm import tqdm

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.eval()
model.to(device)

# Initialize features and labels as empty numpy arrays
X_train = np.array([])
y_train = np.array([])

# Wrap your dataloader with tqdm for a progress bar
for image, target, fname in tqdm(dataloader_train_classifier):
    with torch.no_grad():
        # Forward pass to extract features
        # Note: You might need to modify this depending on the output of your SimCLR model
        feature = model(image.to(device)).cpu().numpy().flatten()
        target = target.cpu().numpy().flatten()

        # If features and labels are empty, assign the first feature and label
        # Else, stack the new feature and label as a new row

        if X_train.size == 0 and y_train.size == 0:
            X_train = feature
            y_train = target
        else:
            X_train = np.vstack((X_train, feature))
            y_train = np.hstack((y_train, target))
            
# Train the SVM classifier
from sklearn import svm
clf = svm.SVC()
clf.fit(X_train, y_train)

from sklearn import svm
from sklearn.metrics import accuracy_score

# Initialize features and labels as empty numpy arrays
X_test = np.array([])
y_test = np.array([])

for image, target, fname in tqdm(dataloader_test):
    with torch.no_grad():
        
        feature = model(image.to(device)).cpu().numpy().flatten()
        target = target.cpu().numpy().flatten()

    if X_test.size == 0 and y_test.size == 0:
        X_test = feature
        y_test = target
    else:
        X_test = np.vstack((X_test, feature))
        y_test = np.hstack((y_test, target))

# Predict labels for test data
X_test_predicted = clf.predict(X_test)

# Calculate accuracy
accuracy = accuracy_score(y_test, X_test_predicted)

print(f"Model accuracy: {accuracy}")


In [ ]:
import tensorboard